In [25]:
%pylab inline
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['text', 'test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
data = pd.read_csv('smsspamcollection/SMSSpamCollection.txt', sep='\t')
data.columns = ('label','text')
data.head()

,label,text
0,ham,Ok lar... Joking wif u oni...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,U dun say so early hor... U c already then say...
3,ham,"Nah I don't think he goes to usf, he lives aro..."
4,spam,FreeMsg Hey there darling it's been 3 week's n...


In [5]:
data.shape

(5571, 2)

Подготовьте для дальнейшей работы два списка: список текстов в порядке их следования в датасете и список соответствующих им меток классов. В качестве метки класса используйте 1 для спама и 0 для "не спама".

In [34]:
labels = data.label.values
text = data.text.values

In [35]:
y = [1 if x =='spam' else 0 for x in labels]
len(y)

5571

Используя sklearn.feature_extraction.text.CountVectorizer со стандартными настройками, получите из списка текстов матрицу признаков X.

In [36]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(text)

In [37]:
X.shape

(5571, 8711)

Оцените качество классификации текстов с помощью LogisticRegression() с параметрами по умолчанию, используя sklearn.cross_validation.cross_val_score и посчитав среднее арифметическое качества на отдельных fold'ах. Установите random_state=2. Параметр cv задайте равным 10. В качестве метрики качества используйте f1-меру. Получившееся качество - один из ответов, которые потребуются при сдаче задания. Ответ округлить до 1 знака после запятой.

In [38]:
LR = LogisticRegression(random_state=2) 

In [45]:
scores =  cross_val_score(LR, X, y, cv=10, scoring='f1')
print (round (np.mean(scores),1))

0.9


А теперь обучите классификатор на всей выборке и спрогнозируйте с его помощью класс для следующих сообщений:

"FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB"

"FreeMsg: Txt: claim your reward of 3 hours talk time"

"Have you visited the last lecture on physics?"

"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$"

"Only 99$"

Прогнозы классификатора (0 - не спам, 1 - спам), записанные через пробел, будут ответом в одном из вопросов ниже.

In [17]:
LR.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
test = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
"FreeMsg: Txt: claim your reward of 3 hours talk time", "Have you visited the last lecture on physics?",
"Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
"Only 99$"] 
y_pred = LR.predict(vectorizer.transform(test))

In [14]:
y_pred

array([1, 1, 0, 0, 0])

Задайте в CountVectorizer параметр ngram_range=(2,2), затем ngram_range=(3,3), затем ngram_range=(1,3). Во всех трех случаях измерьте получившееся в кросс-валидации значение f1-меры, округлите до второго знака после точки, и выпишете результаты через пробел в том же порядке. В данном эксперименте мы пробовали добавлять в признаки n-граммы для разных диапазонов n - только биграммы, только триграммы, и, наконец, все вместе - униграммы, биграммы и триграммы. Обратите внимание, что статистики по биграммам и триграммам намного меньше, поэтому классификатор только на них работает хуже. В то же время это не ухудшает результат сколько-нибудь существенно, если добавлять их вместе с униграммами, т.к. за счет регуляризации линейный классификатор не склонен сильно переобучаться на этих признаках.

In [41]:
vectorizer1 = CountVectorizer(ngram_range=(2,2))
X_1 = vectorizer1.fit_transform(text)
scores_1 =  cross_val_score(LR, X_1, y, cv=10, scoring='f1')
print (round (np.mean(scores_1),2))

0.82


In [40]:
vectorizer2 = CountVectorizer(ngram_range=(3,3))
X_2 = vectorizer2.fit_transform(text)
scores_2 =  cross_val_score(LR, X_2, y, cv=10, scoring='f1')
print (round (np.mean(scores_2),2))

0.73


In [39]:
vectorizer3 = CountVectorizer(ngram_range=(1,3))
X_3 = vectorizer3.fit_transform(text)
scores_3 =  cross_val_score(LR, X_3, y, cv=10, scoring='f1')
print (round(np.mean(scores_3),2))

0.93


Повторите аналогичный п.7 эксперимент, используя вместо логистической регрессии MultinomialNB(). Обратите внимание, насколько сильнее (по сравнению с линейным классификатором) наивный Байес страдает от нехватки статистики по биграммам и триграммам.

По какой-то причине обучение наивного байесовского классификатора через Pipeline происходит с ошибкой. Чтобы получить правильный ответ, отдельно посчитайте частоты слов и обучите классификатор.

In [28]:
clf = MultinomialNB()

In [43]:
scores1 = cross_val_score(clf, X_1, y, cv=10, scoring='f1')
print (round(np.mean(scores1),2))

scores2 = cross_val_score(clf, X_2, y, cv=10, scoring='f1')
print (round(np.mean(scores2),2))

scores3 = cross_val_score(clf, X_3, y, cv=10, scoring='f1')
print (round(np.mean(scores3),2))

0.65
0.38
0.89


Попробуйте использовать в логистической регрессии в качестве признаков Tf*idf из TfidfVectorizer на униграммах. Повысилось или понизилось качество на кросс-валидации по сравнению с CountVectorizer на униграммах? (напишите в файле с ответом 1, если повысилось, -1, если понизилось, и 0, если изменилось не более чем на 0.01). Обратите внимание, что результат перехода к tf*idf не всегда будет таким - если вы наблюдаете какое-то явление на одном датасете, не надо сразу же его обобщать на любые данные.

In [44]:
tfidf = TfidfVectorizer(ngram_range=(1, 1))
X_4 = tfidf.fit_transform(text)
scores_4 = cross_val_score(LR, X_4, y, cv=10, scoring='f1')
print (round(np.mean(scores_4),2))

0.85
